# Highcode Example: Experimentation based on BikeSharing Data

## 0) Setting up Modeva

In [ ]:
## =============================================================
## Install or update packages(recommended to run in Terminal)
## =============================================================
!pip show modeva
# !pip uninstall modeva
#!pip install modeva

In [ ]:
# To get authentication, use the following command: (To get full access please replace the token to your own token)
from modeva.utils.authenticate import authenticate
authenticate(token='eaaa4301-b140-484c-8e93-f9f633c8bacb')

## 1) Data Modules

- Data Loading
- Data Summary
- Exploratory Data Analysis (EDA)
- Data Processing

In [ ]:
## Create an instance of DataSet class
from modeva import DataSet
ds = DataSet()

In [ ]:
## ----------------------------------------------------------------
## Data Loading: a) built-in data; b) user data
## ----------------------------------------------------------------

## a) Load built-in data: "BikeSharing", "CaliforniaHousing", "SimuCredit", "TaiwanCredit"
ds.load("BikeSharing")
ds

In [ ]:
## b) Load user data as pandas dataframe

# import pandas as pd
# data = pd.read_csv("BikeSharing.csv")
# ds = DataSet()
# ds.load_dataframe(data)

In [ ]:
## ----------------------------------------------------------------
## Data Summary: descriptive statistics
## ----------------------------------------------------------------

result = ds.summary()

In [ ]:
## Overall data summary
result.table["summary"]

In [ ]:
## Summary of numerical features
result.table["numerical"]

In [ ]:
## Summary of categorical features
result.table["categorical"]

In [ ]:
## ----------------------------------------------------------------
## Exploratory Data Analysis (EDA): 1d, 2d, 3d, correlation, pca, umap
## ----------------------------------------------------------------

result = ds.eda_1d(feature="cnt", plot_type="density")
result.plot(figsize=(5, 4))
result = ds.eda_1d(feature="cnt", plot_type="histogram")
result.plot(figsize=(5, 4))

In [ ]:
## 2D plots: pair of numerical features
result = ds.eda_2d(feature_x="hr", feature_y="cnt", feature_color="yr",
                   sample_size=300, smoother_order=None)
result.plot(figsize=(5, 4))

In [ ]:
## 2D plots: pair of categorical features
result = ds.eda_2d(feature_x="season", feature_y="workingday")
result.plot(figsize=(5, 4))

In [ ]:
## 2D plots: numerical and categorical features
result = ds.eda_2d(feature_x="season", feature_y="cnt")
result.plot(figsize=(5, 4))

In [ ]:
## 3D Scatter Plot
result = ds.eda_3d(feature_x="hr", feature_y="atemp", feature_z="cnt",
                   feature_color="yr", sample_size=300)
result.plot(figsize=(6, 5))

In [ ]:
## Correlation Heatmap
result = ds.eda_correlation(features=('hr',
                                      'season',
                                      'workingday',
                                      'weathersit',
                                      'windspeed',
                                      'hum',
                                      'cnt'), sample_size=10000)
result.plot(figsize=(5, 5))

In [ ]:
## PCA - Dimension Reduction
result = ds.eda_pca(features=('hr',
                              'season',
                              'workingday',
                              'weathersit',
                              'windspeed',
                              'hum',
                              'cnt'), n_components=5)
result.plot(figsize=(5, 6))

In [ ]:
## ----------------------------------------------------------------
## Data Preprocessing and Feature Engineering
## 
##    ds.impute_missing: missing value imputation
##    ds.scale_numerical: scaling, standardization of numerical features
##    ds.encode_categorical: one-hot encoding or ordinal encoding for categorical features
##    ds.bin_numerical: binning numerical features into discrete bins
##
## Upon calling these functions, no results will be returned. 
## Data processing will be executed by running ds.preprocess(). 
## To reset preprocessing steps, run ds.reset_preprocess().
## ----------------------------------------------------------------

## First of all, reset all preprocessing steps
ds.reset_preprocess()

## a) data imputation
ds.impute_missing()

## b) scaling for numerical features
ds.scale_numerical(method="minmax")

## c) encoding for numerical features
ds.encode_categorical(features=("season", "weathersit", "holiday", "workingday"), method="ordinal")

## d) binning for numerical features
ds.bin_numerical(features=("atemp", ), bins=10, method="uniform")

## e) execute all preprocessing steps
ds.preprocess()

## Display the preprocessed data
ds.data

In [ ]:
## Compare to the raw data
ds.raw_data

In [ ]:
## ----------------------------------------------------------------
## Other Data Processing Functions
## 
##    ds.set_active_features (ds.set_inactive_features): set some features to be active or inactive
##    ds.set_target: set the target feature
##    ds.set_sample_weight: set the sample_weight feature
##    ds.set_feature_type: change the feature type
##    ds.set_task_type: change task type, including "Regression" and "Classification"
##    ds.set_active_samples (set_inactive_samples): set active samples, used for subsampling or outlier removal
##    ds.set_random_split: automatically set train test split (purly random, on (subsampled if exist) "main" data)
##    ds.set_train_idx: manually set training set index
##    ds.set_test_idx: manually set testing set index
## ----------------------------------------------------------------

## a) set inactive features
ds.set_inactive_features(features=['season', 'workingday', 'temp'])

## b) set target feature
ds.set_target(feature="cnt")

## c) set task type
ds.set_task_type('Regression')

## d) change feature types
ds.set_random_split(test_ratio=0.33)

## 2) Model Modules

- Built-in interpretable models: GLM, DecisionTree, GBDT, RandomForest, XGB, LGBM, CatBoost, GAMINet, ReLuDNN, GLMTree, GLMTreeBoost, NeuralTree, MOE
- Model Training
- Model Tuning
- Model Wrapping
- Model Interpretability
- Model Post-hoc Explainability

In [ ]:
## ----------------------------------------------------------------
## Model Training: e.g. LGBM
## ----------------------------------------------------------------

from modeva.models import MoLGBMRegressor
model_lgbm = MoLGBMRegressor(name="LGBM", max_depth=2, n_estimators=100, verbose=-1)
model_lgbm.fit(ds.train_x, ds.train_y.ravel())

In [ ]:
## ----------------------------------------------------------------
## Model Training: Modeva's native MoE model
## ----------------------------------------------------------------

from modeva.models import MoMoERegressor
model_moe = MoMoERegressor(name="MOE", max_depth=2, n_clusters=5, n_estimators=100, verbose=-1)
model_moe.fit(ds.train_x, ds.train_y.ravel())

In [ ]:
## ----------------------------------------------------------------
## Model Tuning: e.g. Random Search
## ----------------------------------------------------------------

from modeva.models.tune import ModelTuneRandomSearch
hyperspace = dict(n_clusters=[2, 3, 4, 5, 6, 7, 8, 9, 10])
hpo = ModelTuneRandomSearch(dataset=ds,
                            model=MoMoERegressor(verbose=-1))
result = hpo.run(param_distributions=hyperspace,
                 n_iter=5,
                 metric="MSE",
                 cv=5,
                 random_state=0)
result.table

In [ ]:
## ----------------------------------------------------------------
## Refit the model using selected hyperparameter
## ----------------------------------------------------------------
import numpy as np
best_param_idx = np.where(result.value["rank_test_MSE"] == 1)[0][0]
model_moe_tuned = MoMoERegressor(**result.value["params"][best_param_idx],
                                 name="MoE-Tuned",
                                 verbose=-1)
model_moe_tuned.fit(ds.train_x, ds.train_y)
model_moe_tuned

In [ ]:
## ----------------------------------------------------------------
## Model Wrapping: e.g. pre-trained Sklearn-style model
## ----------------------------------------------------------------

from xgboost import XGBRegressor
from modeva.models import MoSKLearnRegressor, MoSKLearnClassifier
model_sk = MoSKLearnRegressor(estimator=XGBRegressor(), name="WrappedXGB") 
model_sk.fit(ds.train_x, ds.train_y.ravel())

In [ ]:
## ----------------------------------------------------------------
## Model Interpretability: 
##    ts.interpret_fi: feature importance
##    ts.interpret_ei: effect importance
##    ts.interpret_local_fi: local feature importance
##    ts.interpret_local_ei: local effect importance
##    ts.interpret_effects: global effect plot
##
## Post-hoc Explainability
##    ts.explain_pfi: permutation feature importance
##    ts.explain_hstatistic: H-statistic for each pair of features
##    ts.explain_pdp: 1D and 2D PDP
##    ts.explain_ale: 1D and 2D ALE
##    ts.explain_lime: LIME for local explanation
##    ts.explain_shap: SHAP for local explanation
## ----------------------------------------------------------------

## Create a TestSuite that bundles dataset and model
from modeva import TestSuite
ts = TestSuite(ds, model_lgbm)

In [ ]:
## Global feature importance and effect importance
result = ts.interpret_fi()
result.plot(figsize=(6, 4))
result = ts.interpret_ei()
result.plot(figsize=(6, 4))

In [ ]:
## Global effect plots
result = ts.interpret_effects(features="hr")
result.plot(figsize=(6, 4))
result = ts.interpret_effects(features=("hr", "atemp"))
result.plot(figsize=(6, 5))

In [ ]:
## Local feature importance and effect importance
result = ts.interpret_local_fi(dataset='test', sample_index=0, centered=True)
result.plot(figsize=(6, 4))
result = ts.interpret_local_ei(dataset='test', sample_index=0)
result.plot(figsize=(6, 4))

In [ ]:
## Post-hoc permutation feature importance
result = ts.explain_pfi()
result.plot(figsize=(6, 4))

In [ ]:
## Post-hoc H-statistic
result = ts.explain_hstatistic(sample_size=1000, grid_resolution=10)
result.plot(figsize=(6, 5))

In [ ]:
## Post-hoc partial dependence plots
result = ts.explain_pdp(features="hr")
result.plot(figsize=(6, 4))
result = ts.explain_pdp(features=("hr", "atemp"))
result.plot(figsize=(6, 5))

In [ ]:
## Post-hoc accumulated local effects
result = ts.explain_ale(features=("hr", "atemp"), dataset="train")
result.plot(figsize=(6, 5))

In [ ]:
## Post-hoc local explainability (LIME and SHAP)
result = ts.explain_lime(dataset="test", sample_index=0, centered=False)
result.plot(figsize=(6.5, 4))
result = ts.explain_shap(dataset="test", sample_index=0)
result.plot(figsize=(6.5, 4))

## 3) Test Modules

- Tests for a single model
- Slicing diagnostics
- Model benchmarking
- Fairness Tests

In [ ]:
## ----------------------------------------------------------------
## Tests for a single model:
##    ts.diagnose_accuracy_table
##    ts.diagnose_residual_analysis
##    ts.diagnose_residual_interpret
##    ts.diagnose_residual_cluster
##    ts.diagnose_reliability
##    ts.diagnose_robustness
##    ts.diagnose_resilience
## ----------------------------------------------------------------

## Performance metrics
result = ts.diagnose_accuracy_table(train_dataset="train", test_dataset="test", metric=None)
result.table

In [ ]:
## Residual analysis
result = ts.diagnose_residual_analysis(features="hr", dataset="test")
result.plot(figsize=(6, 4))

In [ ]:
## Residual interpret
result = ts.diagnose_residual_interpret(dataset="test")
result.plot(figsize=(6, 4))

In [ ]:
## Residual analysis
result = ts.diagnose_residual_cluster(dataset="test")
result.plot(figsize=(6, 4))

In [ ]:
## Reliability (prediction set for binary classification; prediction interval for regression)
result = ts.diagnose_reliability(train_dataset="test", test_dataset="test",
                                 test_size=0.5, random_state=0)
result.plot(figsize=(6, 4))

In [ ]:
## Robustness 
result = ts.diagnose_robustness(dataset="test", perturb_features=None, 
                                noise_levels=(0.2, 0.4, 0.6, 0.8), metric="MAE")
result.plot(figsize=(6, 4))

In [ ]:
# Resilience
result = ts.diagnose_resilience(method="worst-sample", metric="MSE")
result.plot(figsize=(6, 4))

In [ ]:
## ----------------------------------------------------------------
## Slicing-based tests:
##    ts.diagnose_slicing_fi
##    ts.diagnose_slicing_accuracy
##    ts.diagnose_slicing_overfit
##    ts.diagnose_slicing_reliability
##    ts.diagnose_slicing_robustness
##    ts.diagnose_slicing_fairness
## ----------------------------------------------------------------

result = ts.diagnose_slicing_accuracy(features=(("hr",), ("atemp", ), ), metric="MAE",
                                      method="quantile", threshold=None)
result.table

In [ ]:
result = ts.diagnose_slicing_accuracy(features=("hr", "atemp"), method="uniform", bins=10,
                                      metric="MAE", threshold=0.15)
result.plot(figsize=(6, 5))

In [ ]:
result = ts.diagnose_slicing_overfit(train_dataset="train", test_dataset="test",
                                     features="hr", metric="MAE", threshold=None)
result.plot(figsize=(6, 5))

In [ ]:
## ----------------------------------------------------------------
## Model bencharmking/comparison tests:
##    tsc.compare_accuracy_table
##    tsc.compare_robustness
##    tsc.compare_reliability
##    tsc.compare_resilience
##    tsc.compare_slicing_accuracy
##    tsc.compare_slicing_overfit
##    tsc.compare_slicing_robustness
##    tsc.compare_slicing_reliability
## ----------------------------------------------------------------

## create TestSuite that bundles dataset and multiple models
tsc = TestSuite(ds, models=[model_lgbm, model_moe, model_moe_tuned, model_sk])

In [ ]:
result = tsc.compare_accuracy_table(train_dataset="train", test_dataset="test", 
                                    metric=("MAE", "R2"))
result.plot(figsize=(5, 4))

In [ ]:
result = tsc.compare_reliability(train_dataset='test', test_dataset='test',
                                 test_size=0.5, alpha=0.1)
result.plot(figsize=(6, 4))

In [ ]:
result = tsc.compare_resilience(dataset='test', metric="MAE", method='worst-sample')
result.plot(figsize=(6, 4))

In [ ]:
result = tsc.compare_robustness(perturb_features=("hr", "atemp", ), noise_levels=(0.2, 0.4, 0.6, 0.8), 
                                perturb_method="quantile", metric="MAE")
result.plot(figsize=(6, 4))

In [ ]:
result = tsc.compare_slicing_accuracy(features="hr", method="uniform", bins=5, metric="MAE")
result.plot(figsize=(6, 5))

In [ ]:
result = tsc.compare_slicing_overfit(features="hr", method="uniform", bins=5, metric="MAE")
result.plot(figsize=(6, 5))

In [ ]:
result = tsc.compare_slicing_robustness(features="hr", method="uniform", bins=5, 
                                        perturb_features=("hr", "atemp", ),
                                        noise_levels=0.2, 
                                        perturb_method="quantile", metric="MAE")
result.plot(figsize=(6, 4))

In [ ]:
result = tsc.compare_slicing_reliability(features="hr", method="uniform", bins=5, 
                                         train_dataset='test', test_dataset='test',
                                         test_size=0.5, alpha=0.1)
result.plot(figsize=(6, 4))